In [1]:
from pysat.solvers import Glucose3
from itertools import combinations
import numpy as np

**HÀM CHUYỂN ĐỔI**

In [2]:
def to_var(i, j,m):
    return i * m + j + 1

TÌM CÁC LÂN CẬN

In [3]:
def get_neighbors(i, j, n, m):
    neighbors = []
    for x in range(i-1, i+2):
        for y in range(j-1, j+2):
            if x < 0 or x >= n or y < 0 or y >= m:
                continue
            neighbors.append(to_var(x,y,m))
    return neighbors

Hàm để phát sinh các mệnh đề CNF

In [4]:
def generate_clauses(n, m, matrix):
    clauses = []
    for i in range(n):
        for j in range(m):
            # Ràng buộc một ô phải được tô màu đỏ hoặc xanh
            clauses.append([to_var(i,j,m),-(to_var(i, j,m))])
            # Ràng buộc để các ô lân cận ô 0 phải là màu đỏ
            if(isinstance(matrix[i][j],int) and matrix[i][j]==0):
                neighbors=get_neighbors(i,j,n,m)
                for neighbor in neighbors:
                     clauses.append([-neighbor])
            # Ràng buộc để các ô lân cận ô >0 phải có số ô lân cận màu xanh bằng giá trị tại ô đó
            if isinstance(matrix[i][j], int) and matrix[i][j]>0:
                neighbors = get_neighbors(i,j,n,m) 
                combo=list(combinations(neighbors, matrix[i][j]))
                for a in combo:
                    neighbor_copy=neighbors.copy()
                    for x in a:
                        neighbor_copy.remove(x)
                    combo2=neighbor_copy  
                    for x in a:
                        clauses.append([int(h) for h in combo2+[x]])
                    for y in combo2:
                        temp=[-v for v in a]+[-y]
                        clauses.append([int(g) for g in temp])
    return clauses

GIẢI BÀI TOÁN

In [5]:
def to_color(x):
    if x > 0:
        return True # màu đỏ
    else:
        return False # màu xanh
# Hàm để giải quyết bài toán bằng Glucose3
def solve(n, m, matrix):
    clauses = generate_clauses(n, m, matrix)
    g = Glucose3()
    
    # Thêm các mệnh đề CNF vào Glucose3
    for clause in clauses:
        g.add_clause(clause)
        
    # Giải quyết bài toán và trả về kết quả
    if g.solve():
        model = g.get_model()
        return [[to_color(model[i*m+j]) for j in range(m)] for i in range(n)]
    else:
        return None

**TEST**

In [6]:
# Sử dụng ví dụ input.txt
with open('input.txt', 'r') as f:
    lines = f.readlines()
    m ,n= lines[0].strip().split()
    m=int(m)
    n=int(n)
    matrix = []
    for i in range(1,len(lines)):
        row = []
        for c in lines[i].strip().split():
            if c == '.':
                row.append('.')
            else:
                row.append(int(c))
        matrix.append(row)
# Giải quyết bài toán và in ra kết quả
result = solve(n, m, matrix)
if result:
    for x in range(0,len(result)):
        for y in range(0,len(result)):
            if(result[x][y]==True):
                print('\033[42m' + str(matrix[x][y]) + '\033[0m', end='')
            else:
                print('\033[41m' + str(matrix[x][y]) + '\033[0m', end='')
        print()
else:
    print("Không tìm thấy cách giải")


.23..0....
....3.2..6
..5.53.574
.4.5.5.6.3
..4.5.6..3
...2.5....
4.1...11..
4.1...1.4.
....6....4
.44....4..
